# Query & analyze

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

lb.settings.species = "human"

In [ ]:
ln.track()

## Inspect the CellMarker registry ![](https://img.shields.io/badge/Access-10b981) 

Inspect your aggregated cell marker registry:

In [ ]:
lb.CellMarker.filter().df()

Search for a marker (synonyms aware):

In [ ]:
lb.CellMarker.search("PD-1").head(2)

Look up markers with auto-complete:

In [ ]:
markers = lb.CellMarker.lookup()

markers.cd14

## Query files by markers ![](https://img.shields.io/badge/Access-10b981) 

Query panels and datasets based on markers, e.g., which datasets have `'CD14'` in the flow panel:

In [ ]:
panels_with_cd14 = ln.FeatureSet.filter(cell_markers=markers.cd14).all()

In [ ]:
ln.File.filter(feature_sets__in=panels_with_cd14).df()

Access registries:

In [ ]:
features = ln.Feature.lookup()
efs = lb.ExperimentalFactor.lookup()
species = lb.Species.lookup()

Find shared cell markers between two files:

In [ ]:
files = ln.File.filter(feature_sets__in=panels_with_cd14, species=species.human).list()
file1, file2 = files[0], files[1]

In [ ]:
shared_markers = file1.features["var"] & file2.features["var"]
shared_markers.list("name")

## Concatenate & analyze queried files ![](https://img.shields.io/badge/Transform-10b981)

Load files into memory and concatenate:

In [ ]:
adata1 = file1.load()
adata2 = file2.load()

In [ ]:
import anndata as ad

In [ ]:
adata = ad.concat(
    [adata1, adata2],
    label="file",
    keys=[file1.description, file2.description],
)
adata

In [ ]:
import scanpy as sc

sc.pp.pca(adata)
sc.pl.pca(adata, color=markers.cd14.name)

## Register a concatenated dataset ![](https://img.shields.io/badge/Register-10b981)

If we believe that we'll need this dataset, again, we can register a concatenated version:

In [ ]:
dataset = ln.Dataset(adata, name="Concatenated dataset")
dataset.save()
dataset.view_flow()

In [ ]:
# clean up test instance
!lamin delete --force test-flow
!rm -r test-flow